`python` 에서 수학 관련 기능을 좀 더 쉽게 사용할 수 있도록 해 주는 `module`을 불러 들임

In [ ]:
import numpy as np
import numpy.linalg as na
import matplotlib.pyplot as plt
import sympy as sy
import IPython.display as disp
sy.init_printing()

## 예제 03.001

### 문제에서 주어진 매개변수

In [ ]:
subs_dict = {'L_AB_m': 2.5, # A-B 구간의 길이
             'L_BC_m': 1.5, # B-C 구간의 길이
             'd_mm': 60.0, # 실린더 지름
             'T_A_Nm': 3600.0, # A 지점에서의 비틀림 하중
             'T_B_Nm': -1600.0, # B 지점에서의 비틀림 하중
             'G_Pa': 80e9, # 탄성계수
            }
subs_dict['d_m'] = subs_dict['d_mm'] * 1e-3 # 실린더 지름

In [ ]:
L_AB_m, L_BC_m, d_m, T_A_Nm, T_B_Nm, G_Pa = sy.symbols('L_AB_m, L_BC_m, d_m, T_A_Nm, T_B_Nm, G_Pa')

In [ ]:
d_mm = d_m * 1e3

### 1) 각 원통 A-B, B-C 에서의 최대 응력

#### C점에서의 반력

C 점에서의 반력을 `R_C` 라고 정한다.

In [ ]:
R_C_Nm = sy.symbols('R_C_Nm')

평형을 유지하기 위해 A, B, C 지점에서의 비틀림 하중과 반력을 모두 합하면 0이 될 것이다.
$$T_A + T_B + R_C = 0$$
이것을 python 으로는 다음과 같이 표현할 수 있다.

In [ ]:
eq_eq = sy.Eq(T_A_Nm + T_B_Nm + R_C_Nm, 0)

이 식을 풀면 반력을 구할 수 있다. (아래에서 `_` 는 바로 위 결과를 말함)

In [ ]:
R_C_Sol = sy.solve(eq_eq, R_C_Nm)
disp.display(R_C_Sol)

#### 반지름과 단면의 극관성모멘트

반지름 r 은 다음과 같이 구할 수 있다.

In [ ]:
r_m = d_m * 0.5

원형 단면의 비틀림인 경우 극관성모멘트를 구한다.

In [ ]:
J_m4 = (sy.pi * d_m**4) / 32

#### 최대전단응력

A-B 구간에서의 최대전단응력은 다음과 같이 구할 수 있을 것이다.

In [ ]:
tau_max_AB_Pa = T_A_Nm * r_m / J_m4

B-C 구간에서의 최대전단응력도 마찬가지로 구할 수 있다.

In [ ]:
tau_max_BC_Pa = (T_A_Nm + T_B_Nm) * r_m / J_m4
disp.display(sy.simplify(tau_max_BC_Pa))

각 변수에 값을 대입하면 다음과 같다.

In [ ]:
disp.display(disp.Math('$\\tau_{maxAB}=%g(Pa)$'%float(tau_max_AB_Pa.subs(subs_dict))))
disp.display(disp.Math('$\\tau_{maxAB}=%g(MPa)$'%(1e-6*float(tau_max_AB_Pa.subs(subs_dict)))))

In [ ]:
disp.display(disp.Math('$\\tau_{maxBC}=%g(Pa)$'%float(tau_max_BC_Pa.subs(subs_dict))))
disp.display(disp.Math('$\\tau_{maxBC}=%g(MPa)$'%(1e-6*float(tau_max_BC_Pa.subs(subs_dict)))))

### 2) 끝점 A 에서의 변형량

축방향 하중의 경우 변형량은 아래와 같았다.
$$\delta=\frac{PL}{EA}$$

비틀림 하중의 경우 변형량은 다음과 같다.
$$\theta = \frac{TL}{GJ}$$

A-B 구간, B-C 구간의 변형량은 각각 다음과 같다.

In [ ]:
theta_AB = T_A_Nm * L_AB_m / (G_Pa * J_m4)
disp.display(theta_AB)

In [ ]:
theta_BC = (T_A_Nm + T_B_Nm) * L_BC_m / (G_Pa * J_m4)
disp.display(theta_BC)

각각 변수 값을 대입 하면 다음과 같다.

In [ ]:
disp.display(disp.Math('$\\theta_{AB}=%g(rad)$'%float(theta_AB.subs(subs_dict))))
disp.display(disp.Math('$\\theta_{AB}=%g(deg)$'%(float((180/sy.pi)*theta_AB.subs(subs_dict)))))

In [ ]:
disp.display(disp.Math('$\\theta_{BC}=%g(rad)$'%float(theta_BC.subs(subs_dict))))
disp.display(disp.Math('$\\theta_{BC}=%g(deg)$'%(float((180/sy.pi)*theta_BC.subs(subs_dict)))))

전체 변형량은 다음과 같다.

In [ ]:
theta_AC = sy.simplify(theta_AB + theta_BC)
disp.display(theta_AC)

In [ ]:
disp.display(disp.Math('$\\theta_{AC}=%g(rad)$'%float(theta_AC.subs(subs_dict))))
disp.display(disp.Math('$\\theta_{AC}=%g(deg)$'%(float((180/sy.pi)*theta_AC.subs(subs_dict)))))

### 1) 각 원통 A-B, B-C 에서의 최대 응력 : 그림으로 표시해 보자

C 점의 x 좌표를 0으로 하자. C-B, B-A 각 구간을 다음과 같이 준비할 수 있다.

In [ ]:
x_CB_m_array = np.linspace(0, subs_dict['L_BC_m'], 101)
x_BA_m_array = np.linspace(0, subs_dict['L_AB_m'], 101) + subs_dict['L_BC_m']

각 구간 안의 x 좌표의 수는 아래와 같이 알 수 있다.

In [ ]:
(x_CB_m_array.shape, x_BA_m_array.shape)

두 구간을 다음과 같이 하나로 합칠 수도 있다.

In [ ]:
x_m_array = np.hstack((x_CB_m_array, x_BA_m_array))
x_m_array.shape

B-A 구간에서는 $T_{A}(Nm)$ 이 작용하고 있다. 위에서 정한 B-A 구간의 x 좌표 수와 같은 수의 1 을 담은 배열 `array` 을 만든 후 $T_A(Nm)$ 을 곱한다.

In [ ]:
T_BA_Nm_array = np.ones_like(x_BA_m_array) * subs_dict['T_A_Nm']

C-B 구간에서는 $T_{B}(Nm)$ 이 작용하고 있으므로 이번에도 위에서 정한 C-B 구간의 x 좌표 수와 같은 수의 1 을 담은 배열 `array` 을 만든 후 $T_B(Nm)$ 을 곱한다.

In [ ]:
T_CB_Nm_array = np.ones_like(x_CB_m_array) * (subs_dict['T_A_Nm'] + subs_dict['T_B_Nm'])

이렇게 준비한 배열을 그래프로 표시하되, 선도 아래 부분이 칠해지도록 `fill_between()` 기능을 이용해본다.

In [ ]:
plt.clf()  # 이전에 그려진 그림이 있었다면 지우고 새로 시작한다.
plt.fill_between(x_CB_m_array, T_CB_Nm_array)
plt.fill_between(x_BA_m_array, T_BA_Nm_array)

A, B, C 점을 표시한다.

In [ ]:
plt.text(x_CB_m_array[0], 0, 'C')
plt.text(x_BA_m_array[0], 0, 'B')
plt.text(x_BA_m_array[-1], 0, 'A')

x y 축 제목을 붙인다.

In [ ]:
plt.xlabel('x(m)')
plt.ylabel('T(Nm)')

In [ ]:
plt.grid(True)
plt.show()